# Discovering DWave Ocean SDK

In [ ]:
#First if not yet done, install the ocean sdk package 
#!pip install dwave-ocean-sdk
#!pip install networkx

You need to https://www.dwavesys.com/build/getting-started/ and connect to your leap account with the following commands run in a terminal (and follow instructions)

```
dwave auth login --oob
dwave auth get
dwave config create --auto-token
```

And the following to be able to use dwave inspector (read the EULA):

```
dwave install inspector
```

# Our first QBM

In this first part, we will solve a constraint scheduling problem as defined here: https://docs.ocean.dwavesys.com/en/stable/examples/scheduling.html

## Problem definition

In [ ]:
from dimod import BinaryQuadraticModel
# build directly from a QUBO matrix (equivalent)
Q={(0,0): 1, (1,1): 1, (2,2):-1, (0,1): -2, (0,2): 1, (0,3): -1}
bqm = BinaryQuadraticModel.from_qubo(Q, offset=1)
print("create from QUBO matrix", bqm)
# or create from variables
bqm = BinaryQuadraticModel({'t': 1, 'v': 1, 'l': -1},      # Linear terms
                           {'tv': -2, 'tl': 1, 'tp': -1},  # Quadratic terms
                           1,                              # Offset
                           'BINARY')                       # 'SPIN' or 'BINARY'
print("create from variables", bqm)

## Solvers

In [ ]:
from dimod.reference.samplers import ExactSolver
# Exact Solver - https://docs.ocean.dwavesys.com/en/stable/docs_dimod/reference/sampler_composites/samplers.html#dimod.reference.samplers.ExactSolver
sampler = ExactSolver()
# will sample all the possible states
sampleset = sampler.sample(bqm)

In [ ]:
print(sampleset)

In [ ]:
print(sampleset.lowest(atol=.5))

In [ ]:
# Classical solvers
# Random Sampler - https://docs.ocean.dwavesys.com/en/stable/docs_samplers/reference.html#dwave.samplers.RandomSampler
from dwave.samplers import RandomSampler
random_sampler = RandomSampler()
random_sampleset = random_sampler.sample(bqm, num_reads=5)
print(random_sampleset)
print(random_sampleset.lowest(atol=.5))

In [ ]:
# Simulated Annealing - https://docs.ocean.dwavesys.com/en/stable/docs_samplers/reference.html#dwave.samplers.SimulatedAnnealingSampler
from dwave.samplers import SimulatedAnnealingSampler
sa_sampler = SimulatedAnnealingSampler()
sa_sampleset = sa_sampler.sample(bqm)
print(sa_sampleset.lowest(atol=.5))

In [ ]:
for sample, energy in sampleset.data(['sample', 'energy']):  
    if energy==0:
        time = 'business hours' if sample['t'] else 'evenings'
        venue = 'office' if sample['v'] else 'home'
        length = 'short' if sample['l'] else 'long'
        participation = 'mandatory' if sample['p'] else 'optional'
        print("During {} at {}, you can schedule a {} meeting that is {}".format(time, venue, length, participation))

## Using a QPU sampler

Before doing that, you need to evaluate the time in will take out of your credits

In [ ]:
from dwave.system import DWaveSampler, EmbeddingComposite

qpu = DWaveSampler()
print(f"time estimation for 100 qubits, 1000 samples {qpu.solver.estimate_qpu_access_time(100, 1000)}us on {qpu.solver.name}")

In [ ]:
# Then solve the problem on QPU
# Embedding Composite is doing the minor embedding for you
sampler = EmbeddingComposite(DWaveSampler())
sampleset = sampler.sample(bqm, num_reads=5000, label='SDK Examples - Scheduling')

In [ ]:
print(sampleset.lowest(atol=.5))                     

## Map Colouring

We are interested in this notebook: https://docs.ocean.dwavesys.com/en/stable/examples/map_kerberos.html

In [ ]:
import networkx as nx
G = nx.read_adjlist('data/usa.adj', delimiter = ',')   

In [ ]:
states = G.nodes        
borders = G.edges 

In [ ]:
import dimod

colors = [0, 1, 2, 3]

dqm = dimod.DiscreteQuadraticModel()
for state in states:            
   dqm.add_variable(4, label=state)
for state0, state1 in borders:          
   dqm.set_quadratic(state0, state1, {(color, color): 1 for color in colors})

In [ ]:
from dwave.system import LeapHybridDQMSampler

In [ ]:
sampleset = LeapHybridDQMSampler().sample_dqm(dqm,
                label='SDK Examples - Map Coloring DQM')  

print("Energy: {}\nSolution: {}".format(
       sampleset.first.energy, sampleset.first.sample))  

In [ ]:
import matplotlib.pyplot as plt       
node_list = [list(G.nodes)[x:x+10] for x in range(0, 50, 10)]   
node_list[4].append('ND')     
# Adjust the next line if using a different map
nx.draw(G, pos=nx.shell_layout(G, nlist = node_list), with_labels=True,
        node_color=list(sampleset.first.sample.values()), node_size=400,
        cmap=plt.cm.rainbow)                 
plt.show()

# Graph Partitioning
The problem of interest here, which is NP-hard, is to try and find the best division of the graph’s nodes into two equal sets with a minimum number of edges between the two groups.

See https://docs.ocean.dwavesys.com/en/stable/examples/inspector_graph_partitioning.html

In [ ]:
import networkx as nx

graph_nodes = 26
G = nx.random_geometric_graph(n=graph_nodes, radius=.5, dim=2)

In [ ]:
from collections import defaultdict
from itertools import combinations

gamma = 70
Q = defaultdict(int)
# Fill in Q matrix
for u, v in G.edges:
    Q[(u,u)] += 1
    Q[(v,v)] += 1
    Q[(u,v)] += -2
for i in G.nodes:
    Q[(i,i)] += gamma*(1-len(G.nodes))
for i, j in combinations(G.nodes, 2):
    Q[(i,j)] += 2*gamma

In [ ]:
print("Maximum element is {:.2f} and minimum is {:.2f}.".format(max(Q.values()), min(Q.values())))    

In [ ]:
import numpy as np
from dwave.system import DWaveSampler, EmbeddingComposite

# Import the problem inspector to begin data capture
import dwave.inspector

sampler = EmbeddingComposite(DWaveSampler())
num_reads = 1000
sampleset = sampler.sample_qubo(Q, num_reads=num_reads, \
                                label='SDK Examples - Inspector') 

In [ ]:
print("Number of nodes in one set is {}, in the other, {}. \nEnergy is {}.".format(
       sum(sampleset.first.sample.values()),
       graph_nodes - sum(sampleset.first.sample.values()),
       sampleset.first.energy))

In [ ]:
dwave.inspector.show(sampleset)   

# Working with PyQUBO

In [ ]:
#!pip install pyqubo

In [ ]:
from pyqubo import Spin
s1, s2, s3, s4 = Spin("s1"), Spin("s2"), Spin("s3"), Spin("s4")
H = (4*s1 + 2*s2 + 7*s3 + s4)**2
model = H.compile()
qubo, offset = model.to_qubo()
print(qubo)

In [ ]:
import neal
sampler = neal.SimulatedAnnealingSampler()
bqm = model.to_bqm()
sampleset = sampler.sample(bqm, num_reads=50)
decoded_samples = model.decode_sampleset(sampleset)
print(sampleset)
best_sample = min(decoded_samples, key=lambda x: x.energy)
best_sample.sample